In [1]:
import requests
import urllib.parse
from src.ParliamentDataHandler import ParliamentDataHandler
import pandas as pd

In [2]:
handler = ParliamentDataHandler()

# MEPs

In [5]:
api_endpoint = "https://api.europarl.europa.eu/documents/mep/{mep_id}"

# Replace with the MEP's identifier
mep_id = "person/1"

# Make the API request
response = requests.get(api_endpoint.format(mep_id=mep_id))


ConnectionError: HTTPSConnectionPool(host='api.europarl.europa.eu', port=443): Max retries exceeded with url: /documents/mep/person/1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000275DC8D0410>: Failed to resolve 'api.europarl.europa.eu' ([Errno 11001] getaddrinfo failed)"))

# Meetings

In [4]:
# Initialize start and end dates
init_date = '01/01/2024'
end_date = '14/10/2024'

# Execute the batch scraping with concurrency
meetings = handler.scrape_meetings_in_parallel(init_date, end_date)


df_new = pd.DataFrame(meetings)
df_meetings = pd.read_csv('./data/meetings.csv')
pd.concat([df_meetings, df_new]).drop_duplicates().to_csv('./data/meetings.csv', index=False)

10it [00:04,  2.38it/s]


# Questions

In [22]:
questions = handler.get_questions()

An error occurred: URL - https://data.europarl.europa.eu/api/v2/parliamentary-questions?format=application%2Fld%2Bjson&offset=0&limit=1000 ERROR: Expecting value: line 1 column 1 (char 0)


In [ ]:
from tqdm import tqdm
import concurrent.futures
import time

def fetch_question_details(identifiers):
    results = []
    for identifier in identifiers:
        if identifier:
            results.append(handler.get_questions_details(identifier))
    time.sleep(1) # Add 1 second sleep after each batch
    return results

# Split questions into batches of 10
batch_size = 10
question_batches = []
current_batch = []

for q in quetsions:
    identifier = q.get("identifier")
    if len(current_batch) < batch_size:
        current_batch.append(identifier)
    else:
        question_batches.append(current_batch)
        current_batch = [identifier]
if current_batch:
    question_batches.append(current_batch)

final = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(fetch_question_details, batch) for batch in question_batches]
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(question_batches)):
        final.extend(future.result())


  0%|          | 0/1 [00:00<?, ?it/s]

An error occurred: URL - https://data.europarl.europa.eu/api/v2/parliamentary-questions/E-10-2024-002309?format=application%2Fld%2Bjson&language=en ERROR: Expecting value: line 1 column 1 (char 0)
An error occurred: URL - https://data.europarl.europa.eu/api/v2/parliamentary-questions/E-10-2024-002429?format=application%2Fld%2Bjson&language=en ERROR: Expecting value: line 1 column 1 (char 0)
An error occurred: URL - https://data.europarl.europa.eu/api/v2/parliamentary-questions/E-10-2024-002308?format=application%2Fld%2Bjson&language=en ERROR: Expecting value: line 1 column 1 (char 0)
An error occurred: URL - https://data.europarl.europa.eu/api/v2/parliamentary-questions/E-10-2024-002064?format=application%2Fld%2Bjson&language=en ERROR: Expecting value: line 1 column 1 (char 0)
An error occurred: URL - https://data.europarl.europa.eu/api/v2/parliamentary-questions/E-10-2024-002077?format=application%2Fld%2Bjson&language=en ERROR: Expecting value: line 1 column 1 (char 0)
An error occurr

In [14]:
for q in quetsions:
    try:
        q['identifier']
    except:
        print(q)



{'id': 'eli/dl/proc/1995-2618', 'type': 'Process', 'process_id': '1995-2618', 'process_type': 'def/ep-procedure-types/RSP', 'label': '1995/2618(RSP)'}
{'id': 'eli/dl/proc/1996-0129', 'type': 'Process', 'process_id': '1996-0129', 'process_type': 'def/ep-procedure-types/COD', 'label': '1996/0129(COD)'}
{'id': 'eli/dl/proc/1996-0186', 'type': 'Process', 'process_id': '1996-0186', 'process_type': 'def/ep-procedure-types/CNS', 'label': '1996/0186(CNS)'}
{'id': 'eli/dl/proc/1996-0167', 'type': 'Process', 'process_id': '1996-0167', 'process_type': 'def/ep-procedure-types/CNS', 'label': '1996/0167(CNS)'}
{'id': 'eli/dl/proc/1996-2502', 'type': 'Process', 'process_id': '1996-2502', 'process_type': 'def/ep-procedure-types/RSP', 'label': '1996/2502(RSP)'}
{'id': 'eli/dl/proc/1997-0271', 'type': 'Process', 'process_id': '1997-0271', 'process_type': 'def/ep-procedure-types/CNS', 'label': '1997/0271(CNS)'}
{'id': 'eli/dl/proc/1996-0092', 'type': 'Process', 'process_id': '1996-0092', 'process_type': 